In [12]:
from PIL import Image, ExifTags
import base64
from pathlib import Path
import os
from io import BytesIO

In [13]:
image_path = Path(os.getcwd())/'ExampleImages/20221106_140301.jpg'

In [14]:
def load_file_directly(file_path):
    with open(file_path, "rb") as img_file:
        b64_string = base64.b64encode(img_file.read()).decode('utf-8')
    return b64_string

In [15]:
%timeit load_file_directly(image_path)

13.7 ms ± 63.2 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [24]:
def pil_load_and_resize(file_path, target_size=1024):
    image = Image.open(file_path)
    exif = image._getexif()

    width, height = image.size
    max_dimension = max(width, height)

    factor = 1
    while max_dimension >= target_size:
        factor *= 2
        max_dimension //= 2

    if factor > 1:
        image = image.resize((width // factor, height // factor), resample=Image.LANCZOS)

    for orientation in ExifTags.TAGS.keys():
        if ExifTags.TAGS[orientation] == 'Orientation':
            break

    if exif[orientation] == 3:
        image=image.rotate(180, expand=True)
    elif exif[orientation] == 6:
        image=image.rotate(270, expand=True)
    elif exif[orientation] == 8:
        image=image.rotate(90, expand=True)

    buffered = BytesIO()
    image.save(buffered, format="JPEG")
    return base64.b64encode(buffered.getvalue()).decode('utf-8')

In [25]:
%timeit pil_load_and_resize(image_path)

D:\Users\berta\AppData\Local\Temp\ipykernel_1832\1337952680.py:14: DeprecationWarning: LANCZOS is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.LANCZOS instead.
  image = image.resize((width // factor, height // factor), resample=Image.LANCZOS)


249 ms ± 1.98 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [26]:
def pil_load(file_path):
    image = Image.open(file_path)
    exif = image._getexif()

    for orientation in ExifTags.TAGS.keys():
        if ExifTags.TAGS[orientation] == 'Orientation':
            break

    if exif[orientation] == 3:
        image=image.rotate(180, expand=True)
    elif exif[orientation] == 6:
        image=image.rotate(270, expand=True)
    elif exif[orientation] == 8:
        image=image.rotate(90, expand=True)

    buffered = BytesIO()
    image.save(buffered, format="JPEG")
    return base64.b64encode(buffered.getvalue()).decode('utf-8')

In [27]:
%timeit pil_load(image_path)

393 ms ± 10.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [29]:
def check_max_size(file_path, target_size=1024):
    image = Image.open(file_path)
    exif = image._getexif()

    width, height = image.size
    max_dimension = max(width, height)

    return max_dimension >= target_size

In [30]:
%timeit check_max_size(image_path)

846 µs ± 10.9 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


The speed reduction of the resizing is worth it for uploading but loading images straight from PIL involves saving the image to a buffer which is slow. Saving 16 images involves 4 seconds of waiting for the resizing.

It might be worth caching the resized images and that would be very fast. Encrypted images could be uploaded online and cached down-sized ones could be stored on disk.

Checking the image size with PIL is very fast.